In [34]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import boxcox
from precision_recall_cutoff import precision_recall_cutoff
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, RFECV

from sklearn.metrics import recall_score, classification_report

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'turnover.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [35]:
## changing 'sales' and 'salary' to dummy variables
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis = 1), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [36]:
## Engineering Features from the decision tree model
turnover['interaction_1'] = np.where(((turnover['satisfaction_level'] >= 0.115) &
                                      (turnover['satisfaction_level'] <= 0.465) &
                                      (turnover['number_project'] > 2.5)), 1, 0)
                              
turnover['interaction_2'] = np.where(((turnover['satisfaction_level'] <= 0.465) &
                                      (turnover['satisfaction_level'] <= 2.5) &
                                      (turnover['number_project'] <= 0.575)), 1, 0)

turnover['interaction_3'] = np.where(((turnover['satisfaction_level'] > 0.465) &
                                      (turnover['time_spend_company'] <= 0.45) &
                                      (turnover['average_montly_hours'] <= 290.5)), 1, 0)

In [37]:
## defining input and target
x = turnover.drop(columns = 'left', axis = 1)
y = turnover['left']

## splitting the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

### RFECV Random Forest
***

In [38]:
rf_rfe = RFECV(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), 
               step = 1, min_features_to_select = 2, cv = 3).fit(x_train, y_train)

print(x_train.columns[rf_rfe.support_])

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_RandD', 'sales_accounting',
       'sales_management', 'sales_technical', 'salary_high', 'salary_low',
       'salary_medium', 'interaction_1'],
      dtype='object')


In [39]:
x_train_2 = x_train[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_RandD', 'sales_accounting',
       'sales_management', 'sales_technical', 'salary_high', 'salary_low',
       'salary_medium', 'interaction_1']]

x_test_2 = x_test[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_RandD', 'sales_accounting',
       'sales_management', 'sales_technical', 'salary_high', 'salary_low',
       'salary_medium', 'interaction_1']]
    
## building random forest model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(x_train_2, y_train)

## predicting on the testing
rf_preds = rf_md.predict_proba(x_test_2)[:, 1]

## changing likelihoods to labes
rf_labels = precision_recall_cutoff(y_test, rf_preds)

## printing classification report
print(classification_report(y_test, rf_labels))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96      2286
           1       0.86      0.88      0.87       714

    accuracy                           0.94      3000
   macro avg       0.91      0.92      0.91      3000
weighted avg       0.94      0.94      0.94      3000



### GridsearchCV Random Forest
***

In [40]:
## defining hyperparameters to consider tuning
rf_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth': [3, 5, 7]}

## running GridsearchCV
rf_grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = rf_param_grid, 
                              cv = 5, scoring = 'f1').fit(x_train_2, y_train)

## extracting the best hyperparameter combinationsRF_param_grid
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 15,
 'n_estimators': 500}

In [41]:
## building random forest model
rf_md2 = RandomForestClassifier(n_estimators = 500, max_depth = 7,
                               min_samples_leaf = 5, min_samples_split = 15).fit(x_train_2, y_train)

## predicting on the testing
rf_preds2 = rf_md2.predict_proba(x_test_2)[:, 1]

## changing likelihoods to labes
rf_labels2 = precision_recall_cutoff(y_test, rf_preds2)

## printing classification report
print(classification_report(y_test, rf_labels2))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2286
           1       0.98      0.91      0.94       714

    accuracy                           0.97      3000
   macro avg       0.98      0.95      0.96      3000
weighted avg       0.97      0.97      0.97      3000



From the above results, I would choose the second model because it has a higher performance to predict 'left'.